## imports

In [ ]:
#! /usr/bin/env python3

import os
import csv
from collections import namedtuple

from nameparser import HumanName
from pymarc import MARCReader
from titlecase import titlecase

In [ ]:
import os
import openpyxl

## reading source data

In [ ]:
def lookup_uid(record):
    return record.get_fields('001')[0].value().replace('AAI','')

In [ ]:
unrestricteds_filepath = 'source_data/ProquestDissertations/UnrestrictedTheses'

with open(os.path.join(unrestricteds_filepath, 'MARCDATA.MRC'), 'rb') as f:
    reader = MARCReader(f)
    marc_unrestricted_records = list()
    for record in reader:
        marc_unrestricted_records.append(record)

In [ ]:
restricteds_filepath = 'source_data/ProquestDissertations/RestrictedTheses'

with open(os.path.join(restricteds_filepath, 'MARCDATA.MRC'), 'rb') as f:
    reader = MARCReader(f)
    marc_restricted_records = list()
    for record in reader:
        marc_restricted_records.append(record)

In [ ]:
images_filepath= 'source_data/Image_Discs_and_Supplement_Files'

with open(os.path.join(images_filepath, 'MARCDATA.MRC'), 'rb') as f:
    reader = MARCReader(f)
    marc_images_records = list()
    for record in reader:
        marc_images_records.append(record)

In [ ]:
marc_all_records = {i for i in marc_restricted_records}.union(
                    {i for i in marc_unrestricted_records}.union(
                     {i for i in marc_images_records}))

marc_unrestricted_uids = {lookup_uid(i) for i in marc_unrestricted_records}
marc_restricted_uids = {lookup_uid(i) for i in marc_restricted_records}
marc_images_uids = {lookup_uid(i) for i in marc_images_records}
marc_all_uids = {lookup_uid(record) for record in marc_all_records}

In [ ]:
odd_spreadsheets = [os.path.join(root, file)
                    for root, dirs, files in os.walk('source_data/Image_Discs_and_Supplement_Files/')
                    for file in files
                    if os.path.splitext(file)[1] == '.xlsx']

nicks_files = {'a': 'source_data/Image_Discs_and_Supplement_Files/PubNames.xlsx',
               'b': 'source_data/Image_Discs_and_Supplement_Files/DAAP_probably_not_yet_DigitalCommons_uploaded.xlsx',
               'c': 'source_data/Image_Discs_and_Supplement_Files/Masters_Theses_French-Cajun_LA_au-permissions.xlsx',
               'd': 'source_data/Legacy name mainframe query blank depts.xlsx',
               'e': 'source_data/Legacy name mainframe query existing depts.xlsx',
#                'f': '/home/francis/Downloads/gradschool_disstheses_1.xls_Wed_Apr_19_12_11_52_2017part_1.xlsx',
              }

def nick_of_file(filename):
    for nick, file in nicks_files.items():
        if filename == file:
            return nick

def parse_workbook(file):
    wb_dict = dict()
    print(file)
    wb = openpyxl.load_workbook(file)
    for sheetname in wb.sheetnames:
        current_sheet = wb.get_sheet_by_name(sheetname)
        sheet_dict = dict()
        for num, row in enumerate(current_sheet.iter_rows()):
            if num == 0:
                keys = [i.value or num for i in row]
                continue
            values = [i.value for i in row]
            row_dict = {keys[i]: values[i] for i in range(len(keys))}
            sheet_dict[num] = row_dict
        wb_dict[sheetname] = sheet_dict
    return wb_dict

all_odds_dict = dict()
for nick, file in nicks_files.items():
    filename = os.path.split(file)[1]
    all_odds_dict[nick] = parse_workbook(file)


In [ ]:
duplicates_filepath = '/media/francis/U/ProquestDissertations/Theses_and_Dissertations/ProquestDissertations/ETDDuplicates'
restricteds_filepath = '/media/francis/U/ProquestDissertations/Theses_and_Dissertations/ProquestDissertations/RestrictedTheses/'
unrestricteds_filepath = '/media/francis/U/ProquestDissertations/Theses_and_Dissertations/ProquestDissertations/UnrestrictedTheses/'
folder_images = '/media/francis/U/ProquestDissertations/Theses_and_Dissertations/Image Discs and Supplement Files/'

folder_restricted = {file.replace('.pdf', '') 
                    for file in os.listdir(restricteds_filepath)
                    if file.replace('.pdf', '').isnumeric()}
folder_unrestricted = {file.replace('.pdf', '') 
                    for file in os.listdir(unrestricteds_filepath)
                    if file.replace('.pdf', '').isnumeric()}
folder_duplicated = {file.replace('.pdf', '') 
                    for file in os.listdir(duplicates_filepath)
                    if file.replace('.pdf', '').isnumeric()}
folder_images = {file.replace('.pdf', '') 
                    for file in os.listdir(images_filepath)
                    if os.path.splitext(file)[1].lower() == '.pdf'}
folder_all = folder_restricted.union(folder_unrestricted).union(folder_duplicated).union(folder_images)

In [ ]:
print('expected:', 'pass', len(marc_unrestricted_uids), 'pass', len(marc_restricted_uids), len(marc_images_records))
print('observed:', len(folder_all), len(folder_unrestricted), len(folder_duplicated), len(folder_restricted), len(folder_images))

## Adding disciplines

In [ ]:
matches = openpyxl.load_workbook('../ETD_to_DigitalCommons/data/Disciplines for imported documents revised for errors.xlsx')
matches_sheet = matches.get_sheet_by_name('Sheet1')

In [ ]:
matches_dict = set()
for num, row in enumerate(matches_sheet.iter_rows()):
    if num == 0:
        headers = (i.value.replace(' ', '') for i in row)
        Matches = namedtuple('Matches', headers)
        continue
    values = (i.value for i in row)
    item = Matches(*values)
    matches_dict.add(item)

In [ ]:
def match_discipline(obs_dept, obs_degree):
    for item in matches_dict:
        dept, degree, discipline = item.Ifdepartmentequals, item.anddegree_nameequals, item.thendisciplinesis
        if obs_dept == dept:
            if degree and obs_degree == degree:
                return discipline
            if not degree:
                return discipline
    else:
        return ''

## Making the crosswalk

In [ ]:
def make_S3_url(uid):
    return 'https://s3-us-west-2.amazonaws.com/setq-digitalcommons/{}.pdf'.format(uid)

In [ ]:
def lookup_clean_title(record):
    text = record.get_fields('245')[0].value()
    return clean_title(text)
    
def clean_title(text):
    text = titlecase(text)
    text = text.replace(':  ', ": ")
    for k, v in wrong_roman_numeral.items():
        if k in text:
            text = text.replace(k, v)
    return text

In [ ]:
wrong_roman_numeral = {' Ii': ' II',
                       ' Iii ': ' III ',
                       '-Iii': '-III',
                       ' Iii.': ' III.',
                       ' Iv ': ' IV ',
                       ' Vi ': ' VI ',
                       ' Iv.': ' IV.',
                       ' Iv)': 'IV)',
                       ' Viii': ' VIII',
                       '-Vii ': '-VII',
                       '-Viii': '-VIII',
                       ' Vii': ' VII',
                       }

In [ ]:
abbr_degree = {"MPT": "Master of Physical Therapy (MPT)",
    "MUP": "Master of Urban Planning (MUP)",
    "DM": "Doctor of Music (DM)",
    "MTS": "Master of Theological Studies (MTS)",
    "AuD": "Doctor of Audiology (AuD)",
    "MSEE": "Master of Science in Electrical Engineering (MSEE)",
    "MSIB": "Master of Science in International Business (MSIB)",
    "MCSM": "Master of Construction Science and Management (MCSM)",
    "PsyD": "Doctor of Psychology (PsyD)",
    "MSEM": "Master of Science in Engineering Management (MSEM)",
    "MSMSE": "Master of Science in Materials Science and Engineering (MSMSE)",
    "RhD": "Doctor of Rehabilitation (RhD)",
    "MATE": "Master of Arts in the Teaching of English (MATE)",
    "DPT": "Doctor of Physical Therapy (DPT)",
    "MSAgE": "Master of Science in Agricultural Engineering (MSAgE)",
    "PhDOtol": "PhD Otolaryngology (PhDOtol)",
    "MSHRM": "Master of Science in Human Resources Management (MSHRM)",
    "MIM": "Master of International Management (MIM)",
    "DMin": "Doctor of Ministry (DMin)",
    "MSIE": "Master of Science in Industrial Engineering (MSIE)",
    "MSISE": "Master of Science in Infrastructure Systems Engineering (MSISE)",
    "DPA": "Doctor of Public Administration (DPA)",
    "HSOP": "Doctor of Philosophy in Health Services Research (HSOP)",
    "MMatSE": "Master of Materials Science and Engineering (MMatSE)",
    "MAeroE": "Master of Aeronautical Engineering (MAeroE)",
    "MMT": "Master in Management of Technology (MMT)",
    "MSJ": "Master of Science in Jurisprudence (MSJ)",
    "MHP": "Master of Historic Preservation (MHP)",
    "DEng": "Doctor of Engineering (DEng)",
    "MBA": "Master of Business Administration (MBA)",
    "MRED": "Master of Real Estate Development (MRED)",
    "MCTE": "Master of Career and Technology Education (MCTE)",
    "MSAeroE": "Master of Science in Aerospace Engineering (MSAeroE)",
    "MAR": "Master of Arts in Religion (MAR)",
    "MST": "Master's of Science in Teaching (MST)",
    "MJS": "Master of Judicial Studies (MJS)",
    "MALA": "Master of Arts in Liberal Arts (MALA)",
    "MSETM": "Master of Science in Environmental Technology Management (MSETM)",
    "MSHTM": "Master of Science in Hospitality and Tourism Management (MSHTM)",
    "Th.M": "Master of Theology (Th.M)",
    "MSM": "Master of Science in Management (MSM)",
    "MCRP": "Master of City and Regional Planning (MCRP)",
    "MBS": "Master of Building Science (MBS)",
    "MAIS": "Master of Arts in Interdisciplinary Studies (MAIS)",
    "DBA": "Doctor of Business Administration (DBA)",
    "MPH": "Master of Public Health (MPH)",
    "MIDS": "Master of Interdisciplinary Studies (MIDS)",
    "MPA/JD": "Master of Public Administration/Juris Doctorate (MPA/JD)",
    "PhD": "Doctor of Philosophy (PhD)",
    "DMgt": "Doctor of Management (DMgt)",
    "MCIS": "Master of Computer and Information Science (MCIS)",
    "MAE": "Master of Arts in Education (MAE)",
    "MHD": "Master of Human Development (MHD)",
    "MM": "Master of Music (MM)",
    "MGS": "Master of General Studies (MGS)",
    "MSN": "Master of Science in Nursing (MSN)",
    "M.Div": "Master of Divinity (M.Div)",
    "MAC": "Master of Arts in Counseling (MAC)",
    "MCJ": "Master of Criminal Justice (MCJ)",
    "MFR": "Master of Forest Resources (MFR)",
    "MSSS": "Master of Science in Computer Science (MSCS)",
    "MSA": "Master of Science in Administration (MSA)",
    "MURP": "Master of Urban and Regional Planning (MURP)",
    "MAS": "Master in Advanced Studies (MAS)",
    "ND": "Doctor of Nursing (ND)",
    "ME": "Master of Engineering (ME)",
    "MSCRP": "Master of Science in Community and Regional Planning (MSCRP)",
    "MArch": "Master of Architecture (MArch)",
    "MLIS": "Master of Library and Information Science (MLIS)",
    "MSOtol": "MS Otolaryngology (MSOtol)",
    "MLS": "Master of Library Science/Master of Life Sciences (MLS)",
    "MSMANFE": "Master of Science in Manufacturing Engineering (MSMANFE)",
    "MSSE": "Master of Science and Software Engineering (MSSE)",
    "MEngr": "Master of Engineering (MEngr)",
    "MSB": "Masters of Science in Bioscience (MSB)",
    "PED": "Doctor of Physical Education (PED)",
    "MFA": "Master of Fine Arts (MFA)",
    "MMC": "Master of Mass Communication (MMC)",
    "MSBAE": "Master of Science in Biological and Agricultural Engineering (MSBAE)",
    "MAgEd": "Master of Agricultural Education (MAgEd)",
    "MSECE": "Master of Science in Electrical and Computer Engineering (MSECE)",
    "DMD": "Doctor of Dental Medicine (DMD)",
    "MSMatSE": "Master of Science in Material Science Engineering (MSMatSE)",
    "MAPC": "Master of Arts in Pastoral Counseling (MAPC)",
    "MSEd": "Master of Science in Education (MSEd)",
    "DPDS": "Doctor of Planning and Development Studies (DPDS)",
    "MRP": "Master of Regional Planning (MRP)",
    "MNS": "Master of Natural Sciences (MNS)",
    "EdD": "Doctor of Education (EdD)",
    "DrPH": "Doctor of Public Health (DrPH)",
    "DNS": "Doctor of Nursing Science (DNS)",
    "MSIEOR": "Master of Science in Industrial Engineering and Operations Research (MSIEOR)",
    "MAT": "Master of Arts in Teaching (MAT)",
    "MEE": "Master of Electrical Engineering (MEE)",
    "MS": "Master of Science (MS)",
    "MSECO": "Master of Science in Economics (MSECO)",
    "MLA": "Master of Landscape Architecture (MLA)",
    "PhDSurg": "PhD Surgergy (PhDSurg)",
    "MSES": "Master of Science in Engineering Science (MSES)",
    "MHI": "Masters of Health Informatics (MHI)",
    "MSME": "Master of Science in Mechanical Engineering (MSME)",
    "MMUS": "Master of Music (MMUS)",
    "MSW": "Master of Social Work (MSW)",
    "MME": "Master of Music Education (MME)",
    "DMA": "Doctor of Musical Arts (DMA)",
    "MPA": "Master of Public Administration (MPA)",
    "DA": "Doctor of Arts (DA)",
    "MApStat": "Master of Applied Statistics (MApStat)",
    "MSP": "Master of Science in Planning (MSP)",
    "MPP": "Master of Public Policy (MPP)",
    "MSExpSurg": "Medical Surgeon in Experimental Surgery (MSExpSurg)",
    "EdS": "Education Specialist (EdS)",
    "MF": "Master of Forestry (MF)",
    "MPlan": "Master of Planning (MPlan)",
    "MBT": "Master of Business Taxation (MBT)",
    "HSD": "Doctor of Health and Safety (HSD)",
    "MHRD": "Master of Human Resource Development (MHRD)",
    "MSPH": "Master of Science in Public Health (MSPH)",
    "MChE": "Master of Chemical Engineering (MChE)",
    "MSPE": "Master of Science in Petroleum Engineering (MSPE)",
    "MCompE": "Master of Computer Engineering (MCompE)",
    "MT": "Master in Taxation (MT)",
    "MAcc": "Master of Accounting (MAcc)",
    "MPM": "Master of Public Management (MPM)",
    "MSE": "Master of Science in Engineering (MSE)",
    "DME": "Doctor of Music Education (DME)",
    "DSW": "Doctor of Social Work (DSW)",
    "MSCE": "Master of Science in Civil Engineering (MSCE)",
    "DVM": "Doctor of Veterinary Medicine (DVM)",
    "MCE": "Master of Civil Engineering (MCE)",
    "MES": "Master of Environmental Studies (MES)",
    "MECom": "Master of Electronic Commerce (MECom)",
    "MHA": "Master of Health Administration (MHA)",
    "PharmD": "Doctor of Pharmacy (PharmD)",
    "MA": "Master of Arts (MA)",
    "Ded": "Doctor of Education (Ded)",
    "MEnvE": "Master of Environmental Engineering (MEnvE)",
    "ReD": "Doctor of Recreation (ReD)",
    "JD": "Juris Doctorate (JD)",
    "MSBiosyAgE": "Master of Science in Biosystems and Agricultural Engineering (MSBiosyAgE)",
    "PMBA": "Professional Master of Business Administration (PMBA)",
    "MHAMS": "Master of Historical Administration and Museum Studies (MHAMS)",
    "MSIS": "Master of Science in Interdisciplinary Studies (MSIS)",
    "IMES": "International Master of Environmental Sciences (IMES)",
    "MSChE": "Master of Science in Chemical Engineering (MSChE)",
    "MPAcc": "Master of Professional Accounting (MPAcc)",
    "MGIS": "Master of Geographic Information Science (MGIS)",
    "MBioSci": "Master of Biological Science (MBioSci)",
    "MCM": "Master of Construction Management (MCM)",
    "MSMS": "Master of Science in Medical Sciences (MSMS)",
    "MD": "Medical Doctor (MD)",
    "Medical Science": "Doctor of Philosophy (Medical Science)",
    "MGeoE": "Master of Geomechanics Engineering (MGeoE)",
    "MEd": "Master of Education (MEd)",
    "MAM": "Master in Agricultural Management (MAM)",
    "MPRTM": "Master of Parks, Recreation and Tourism Management (MPRTM)",
    "MAgr": "Master of Agriculture (MAgr)",
    "POCS": "Doctor of Oceanography and Coastal Sciences (POCS)",
    "PVMPB": "Doctor of Biomedical and Veterinary Medical Sciences-Pathobiological Sciences (PVMPB)",
    "PNFS": "Doctor of Nutrition and Food Sciences (PNFS)",
    "PENTM": "Doctor of Entomology (PENTM)",
    }



# non_matching_degree_abbrevs = dict()
# def expand_degree_type(degree_name):
#     if degree_name in abbr_degree:
#         return abbr_degree[degree_name]
#     else:
#         if degree_name not in non_matching_degree_abbrevs:
#             non_matching_degree_abbrevs[degree_name] = []
            
# for record in ori:
#     stated_degree = record.get_fields('791')[0].value()
#     stated_degree = stated_degree.replace('.', '')
#     if stated_degree in first_conversion:
#         stated_degree = first_conversion[stated_degree]
#     if stated_degree not in abbr_degree:
#         print(lookup_uid(record))
# #         print(stated_degree, record.get_fields('856')[0].value())

In [ ]:
first_conversion = {'EducatD': 'EdD',
                    'DED': 'EdD',
                    'DMus': 'DMA',
                    'OCS': 'POCS',
                    'VetMedSc': 'PVMPB',
                    'DrSciEng': 'PNFS',
                    'SCDC': 'PENTM'}

def match_degrees(record):
    stated_degree = record.get_fields('791')[0].value()
    if not stated_degree:
        return ''
    alph_degree = stated_degree.replace('.','')
    if alph_degree in first_conversion:
        alph_degree = first_conversion[alph_degree]
    if alph_degree in abbr_degree:
        return abbr_degree[alph_degree]
    else:
        return 'not yet implemented'


In [ ]:
def interpret_directors(record):
    text_a, text_b = parse_500(record)
    return split_directors(text_b)

def parse_500(record):
    value_500 = [i.value() for i in record.get_fields('500')]
    if len(value_500) == 1:
        return value_500[0], ''
    else:
        return value_500[0], value_500[1]  

def split_directors(text_b):
    directors_list = parse_advisors_field(text_b)
    if directors_list:
        if len(directors_list) == 3:
            return directors_list[0], directors_list[1], directors_list[2]
        elif len(directors_list) == 2:
            return directors_list[0], directors_list[1], ''
        elif len(directors_list) == 1:
            return directors_list[0], '', ''
    return ('', '', '')

def parse_advisors_field(text):
    for title in ('Directors: ',
                  'Director: ',
                  'Co-Chairs: ',
                  'Co-chairs: ',
                  'Co-Chairmen: ',
                  'Adviser: ',
                  'Advisers: ',
                  'Chair: ',
                  'Directed: '):
        if title in text:
            text = text.replace(title, '')
            text = text
            text = unperiod(text)
            if text:
                return [i.strip() for i in text.split('; ')]
    else:
        return ''

def unperiod(text):
    if text[-1] == '.':
        return text[:-1]
    return text

In [ ]:
def find_source(record):
    fields = [i.value() for i in record.get_fields('500') if 'Source' in i.value()][0]
    fields = unperiod(fields)
    fields = fields.replace('Source: ', '')
    return fields

In [ ]:
def combine_520(record):
    list_520 = [i for i in record.get_fields('520')]
    if list_520:
        combined_text = ' '.join([i.value() for i in list_520])
    else:
        combined_text = ''
    if combined_text == 'Abstract not available.':
        combined_text = ''
    return combined_text

In [ ]:
def combine_650(record):
    value_650 = [i.value() for i in record.get_fields('650')]
    value_650 = [i.capitalize().replace('.', '') for i in value_650]
    if value_650:
        combined_text = '; '.join(value_650)
    else:
        combined_text = ''
    return combined_text

In [ ]:
def parse_author_names(record):
    name_clump = record.get_fields('100')[0].value()
    name_clump = unperiod(name_clump)
    name = HumanName(name_clump)
    last_name = name.last
    middle_name = name.middle
    suffix = name.suffix
    suffix = standardize_suffix(suffix)
    if name.nickname:
        first_name = "{} {}".format(name.first, name.nickname)
    else:
        first_name = name.first
    return first_name.capitalize(), middle_name.capitalize(), last_name.capitalize(), suffix

def standardize_suffix(text):
    replace_dict = {'JR': 'Jr', 'SR': 'Sr', '3RD': 'III', 'ED': 'Ed.'}
    for wrong in replace_dict:
        if wrong in text:
            text = text.replace(wrong, replace_dict[wrong])
    return text

In [ ]:
def split_institution_department(record, kind='legacy'):
    if record in marc_restricted_records or record in marc_unrestricted_records:
        return (lookup_inst(record), '')
    elif record in marc_images_records:
        inst = 'Louisiana State University and Agricultural & Mechanical College'
        dept = lookup_inst(record).replace('Louisiana State University and Agricultural & Mechanical College', '')
        if dept[:2] == '. ':
            dept = dept[2:]
        if kind == 'current':
            possible_revised_dept = lookup_current_dept(dept)
            if possible_revised_dept:
                dept = possible_revised_dept
        return (inst, dept)

In [ ]:
overlooked_degrees = set()

def read_legacy_dept_map():
    legacy_current = dict()
    sourcepath = '../ETD_to_DigitalCommons/data/LegacyNames.csv'
    with open(sourcepath, encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='\t')
        for num, row in enumerate(csvreader):
            if num == 0:
                headers = (i for i in row)
                continue
            current, legacy = row[0].strip(), row[1].strip()
            if current and current != 'Program no longer active':
                if legacy not in legacy_current:
                    if legacy not in ('New', ):
                        legacy_current[legacy] = current
                else:
                    print(legacy, 'has two mappings')
    return legacy_current

legacy_current = read_legacy_dept_map()
def lookup_current_dept(legacy_dept):
    if legacy_dept in legacy_current:
        return legacy_current[legacy_dept]
    else:
        return None

In [ ]:
def lookup_inst(record):
    text = record.get_fields('710')[0].value()
    text = unperiod(text)
    return text

In [ ]:
def lookup_isbn(record):
    if record.get_fields('020'):
        return record.get_fields('020')[0].value()
    return ''

In [ ]:
def determine_dtype(record):
    for degree in ('PhD', 'DMA', 'EdD', 'DBA', 'PENTM', 'PNFS', 'PVMPB', 'POCS' ):
        if degree in match_degrees(record):
            return "dissertation"
    return "thesis"

In [ ]:
def armageddon_if_restricted(record):
    if record in marc_restricted_records:
        return "9999-12-01"
    return ''

In [ ]:
def is_restricted(record):
    if record in marc_restricted_records:
        return 'withheld'
    elif record in marc_images_records:
        return lookup_odd_permissions(record)
    return 'unrestricted'

In [ ]:
def lookup_odd_permissions(record):
    uids_permissions = {v['PQ_Number']: v['Permissions']
                        for k, v in all_odds_dict['c']['Sheet1'].items()}
    if uids_permissions.get(lookup_uid(record)) == 'denied':
        return 'withheld'
    return 'unrestricted'

In [ ]:
def lookup_mtfc_field_by_uid(uid, field):
    for k, v in all_odds_dict['c']['Sheet1'].items():
        if v['PQ_Number'] == uid:
            sought_value = v.get(field, '')
            if sought_value == 'none':
                return ''
            return sought_value
    return ''

def lookup_DAAP_frompaper_by_uid(uid, field):
    for k, v in all_odds_dict['b']['DigitizeFromPaper'].items():
        if v['PQ_Dig_Num'] == uid:
            return v.get(field, '')
    return ''

## Making the csv

In [ ]:
def csv_writer(data, path):
    with open(path, "w", newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for line in data:
            writer.writerow(line)

In [ ]:
def make_fieldnames_values(record):
    uid = lookup_uid(record)
    fieldnames_values = (('uid', uid),
                         ('title', lookup_clean_title(record)),
                         ('fulltext_url', make_S3_url(uid)),
                         ('keywords', combine_650(record)),
                         ('abstract', combine_520(record)),
                         ('author1_fname', parse_author_names(record)[0]),
                         ('author1_mname', parse_author_names(record)[1]),
                         ('author1_lname', parse_author_names(record)[2]),
                         ('author1_suffix', parse_author_names(record)[3]),
                         ('author1_email', ''),
                         ('author1_institution', split_institution_department(record)[0]),
                         ('advisor1', interpret_directors(record)[0]),
                         ('advisor2', interpret_directors(record)[1]),
                         ('advisor3', interpret_directors(record)[2]),
                         ('availability', is_restricted(record)),
                         ('availability_description', ''),
                         ('comments', ''),
                         ('degree_name', match_degrees(record)),
                         ('department', split_institution_department(record, 'current')[1]),
                         ('legacy_department', split_institution_department(record, 'legacy')[1]),
                         ('disciplines', match_discipline(split_institution_department(record, 'current')[1],
                                                          match_degrees(record))),
                         ('document_type', determine_dtype(record)),
                         ('embargo_date', armageddon_if_restricted(record)),
                         ('publication_date', record.get_fields('792')[0].value()),
                         ('season', ''),
                         ('release_date', ''),
                         ('isbn', lookup_isbn(record)),
                         ('pagelength', record.get_fields('300')[0].value().replace(' p.', '')),
                         ('source', find_source(record)),
                         ('diss_note', unperiod(record.get_fields('502')[0].value())),
                         ('host_item', unperiod(record.get_fields('773')[0].value())),
                         ('language', record.get_fields('793')[0].value()),
#                          ('host_url', record.get_fields('856')[0].value()),
                         ('alternate_uid', lookup_DAAP_frompaper_by_uid(uid, 'Pub_Num')),
                         ('middleton_call_number', lookup_mtfc_field_by_uid(uid, 'MIDL-MAIN_CALL_NO')),
                         ('middleton_location', lookup_mtfc_field_by_uid(uid, 'MIDL-MAIN_LOCATION')),
                         ('docs_micro_call_number', lookup_mtfc_field_by_uid(uid, 'DOCS_MICRO_CALL NO.')),
                         ('docs_micro_location', lookup_mtfc_field_by_uid(uid, 'DOCS_MICRO_LOCATION')),
                         ('specials_call_number', lookup_mtfc_field_by_uid(uid, 'SPEC_COLL_CALL_NO')),
                         ('specials_location', lookup_mtfc_field_by_uid(uid, 'SPEC_COLL_LOCATION')),
                        )
    return fieldnames_values

In [ ]:
def build_csv(records_list, output_folder, output_filename):
    os.makedirs(output_folder, exist_ok=True)
    for num, record in enumerate(records_list):
        if num == 0:
            csv_as_list_of_lists = [[fieldname for fieldname, value in make_fieldnames_values(record)], ]
        record_values = [value for fieldname, value in make_fieldnames_values(record)]
        csv_as_list_of_lists.append(record_values)
    csv_writer(csv_as_list_of_lists, os.path.join(output_folder, output_filename))

In [ ]:
output_filename = 'draft_SetQ.csv'
output_folder = '/home/francis/Desktop/lsu-git/Proquest_to_DigitalCommons/output'

build_csv(marc_images_records, output_folder, output_filename)

## Testing things

In [ ]:
# print a full record matching a specified uid

def find_print_record(uid):
    for record in marc_all_records:
        if lookup_uid(record) == uid:
            return record.as_dict()
        
find_print_record('DP69258')

In [ ]:
# an example of one record

to_do_records = marc_images_records

# marc_images_records[0].as_dict()
find_print_record('DP69224')

In [ ]:
# show all unique values for field 650

all_650a = set()
for record in to_do_records:
    for i in record.get_fields('710'):
        all_650a.add(i.value().replace('Louisiana State University and Agricultural & Mechanical College. ', ''))
#     all_650a.add(record.get_fields('710')[0].value())
print(sorted(all_650a))

In [ ]:
# do any field values have an @ in it?

for record in to_do_records:
    all_fields = [i.value() for i in record.get_fields()]
    for text in all_fields:
        if '@' in text:
            print(text)

In [ ]:
# test of all uids in marc file match a pdf on U drive
# short answer: they all do

pdf_not_on_U = list()

for record in to_do_records:
    uid = lookup_uid(record)
    if os.path.isfile('source_data/Image_Discs_and_Supplement_Files/{}.pdf'.format(uid)):
        continue
    pdf_not_on_U.append(uid)

print(pdf_not_on_U)


In [ ]:
# test of all uids in marc file match a pdf on U drive
# short answer: they all do

# pdf_not_on_U = list()

# for record in marc_images_records:
#     uid = lookup_uid(record)
#     if os.path.isfile('/home/francis/Desktop/lsu-git/Proquest_to_DigitalCommons/source_data/Image Discs and Supplement Files/{}.pdf'.format(uid)):
#         continue
#     pdf_not_on_U.append(uid)

# print(pdf_not_on_U)

In [ ]:
# how many unique values for each field/subfield?

counting_items = dict()

def add_to_if_not_yet(k, v):
    v = v.strip()
    if v == "None" or not v or v == None:
        return
    if k in counting_items:
        counting_items[k].add(v)
    else:
        counting_items[k] = set()
        counting_items[k].add(v)

for record_as_marc in to_do_records:
    record = record_as_marc.as_dict()
    if not record['fields']:
        break
    for dictionary in record['fields']:
        for k, v in dictionary.items():
            if isinstance(v, str) and v:
                add_to_if_not_yet(k, v)
            if isinstance(v, dict) and v:
                ind1 = v['ind1']
                fullpath = '{}/ind1'.format(k)
                add_to_if_not_yet(fullpath, ind1)
                ind2 = v['ind2']
                fullpath = '{}/ind2'.format(k)
                add_to_if_not_yet(fullpath, ind2)
                subfields = v['subfields']
                for subdictionary in subfields:
                    for x, y in subdictionary.items():
                        fullpath = '{}/subfields/{}'.format(k, x)
                        add_to_if_not_yet(fullpath, y)
                        
for k, v in sorted(counting_items.items()):
    print(k, len(v))

In [ ]:
# how many unique values for each field/subfield?

keys_lengths = dict()
all_unique_keys = dict()

def add_to_if_not_yet(dictionary, k, v):
    if k in dictionary:
        dictionary[k].add(v)
    else:
        dictionary[k] = set()
        dictionary[k].add(v)

for record_as_marc in to_do_records:
    record = record_as_marc.as_dict()
    if not record['fields']:
        break        
    field_keys = {k for field in record['fields'] for k in field.keys()}
    fields_list = [k for field in record['fields'] for k in field.keys()]
    for unique_field in field_keys:
        add_to_if_not_yet(keys_lengths, unique_field, fields_list.count(unique_field))
        
for record in to_do_records:
    for field in record.get_fields():
        add_to_if_not_yet(all_unique_keys, field.tag, field.value())

print('this (key) shows up {times} in a record:\n', sorted(keys_lengths.items()))

print('\nthis (key) has {unique values} across the repo:')
for k, v in sorted(all_unique_keys.items()):
    print(k, len(v))

In [ ]:
# all_unique_keys['020']

In [ ]:
# this is supposed to check for broken utf-8, but i don't trust it's working

longest_field = 0

for record_as_marc in to_do_records:
    for field in record_as_marc.get_fields():
        value = field.value()
        try:
            bytes_value = value.encode()
            ascii_value = bytes_value.decode('ascii', "strict")
            if len(ascii_value) > longest_field:
                longest_field = len(ascii_value)
                print(record_as_marc)
        except:
            print(value)

In [ ]:
# find a record with a certain text in any value

for record in marc_unrestricted_records:
    for field in record.get_fields():
        if 'Jalaluddin' in field.value():
            print(lookup_uid(record))

In [ ]:
# for record in marc_unrestricted_records:
#     if lookup_uid(record) in ('7000228', '7009088', '9609119', '9618339') :
#         print(match_degrees(record))

## Cleaning up SetQ sheets / marc

In [ ]:
# How many in the odd set are already in the Uploaded set?

# for file, sheets in all_odds_dict.items():
#     if file == 'a':
#         for sheet, records in sheets.items():
#             for num, record in records.items():
#                 if record['PubNames'] in marc_images_uids:
#                     # print(record['PubNames'])
#                     # no matches
#                     pass
#     if file == 'c':
#         for sheet, records in sheets.items():
#             for num, record in records.items():
#                 if record['PQ_Number'] not in marc_images_uids:
#                     # print(record['PQ_Number'])
#                     # no matches
#                     pass
#     if file == 'b':
#         for sheet, records in sheets.items():
#             if sheet == 'Digitize from paper':
#                 for num, record in records.items():
#                     if record['PQ_Dig_Num'] not in marc_images_uids:
#                         # print(record['PQ_Dig_Num'])
#                         # tons of no matches to uid.
#                         pass
                        


In [ ]:
# in nnnnnnnn format:   DTLSU_Restrictions_uids
#                       DTLSU_from_microfilm_uids
#                       PubNum_to_PQNum.keys()

# in EPnnnnnn format:   PubNum_to_PQNum.values()
#                       MTFC_uids
#                       PubNames_uids


PubNames_uids = {item['PubNames'] for _, item in all_odds_dict['a']['PubNames'].items()}
MTFC_uids = {item['PQ_Number'] for _, item in all_odds_dict['c']['Sheet1'].items()}

PubNum_to_PQNum = {item['Pub_Num']: item['PQ_Dig_Num']
                   for _, item in all_odds_dict['b']['DigitizeFromPaper'].items()}


DTLSU_from_microfilm_uids = {item['Pub_Num']
                             for _, item in all_odds_dict['b']['DigitizeFromMicrofilm'].items()}
DTLSU_Restrictions_uids = {item['Pub_Num']
                           for _, item in all_odds_dict['b']['Restrictions'].items()}


print('DTLSU_Restrictions_uids', '\t\t',len(DTLSU_Restrictions_uids))
print('DTLSU_from_microfilm_uids', '\t\t', len(DTLSU_from_microfilm_uids))
print('PubNum_to_PQNum.keys()', '\t\t\t', len(PubNum_to_PQNum.keys()))
print('PubNum_to_PQNum.values()', '\t\t', len(set(PubNum_to_PQNum.values())))
print('MTFC_uids', '\t\t\t\t', len(MTFC_uids))
print('PubNames_uids', '\t\t\t\t', len(PubNames_uids))


In [ ]:
all_set_Q_nnnnn = DTLSU_Restrictions_uids.union(DTLSU_from_microfilm_uids, PubNum_to_PQNum.keys())
all_set_Q_EPnnn = PubNames_uids.union(PubNum_to_PQNum.values())
all_set_Q_EPnnn.remove(None)

print(len(all_set_Q_nnnnn))
print(len(all_set_Q_EPnnn))

In [ ]:
clean_PubNum_to_PQNum = {k: v for k, v in PubNum_to_PQNum.items()
                         if v}
evil_PubNum_to_PQNum = {k: v for k, v in PubNum_to_PQNum.items()
                        if v not in set(clean_PubNum_to_PQNum.values())}

print(len(clean_PubNum_to_PQNum))
print(len(evil_PubNum_to_PQNum))

In [ ]:
# EPnnnnn format  answer:  unexplained overlap

print(len(PubNames_uids.intersection(MTFC_uids)))                     #40  all match
print(len(PubNames_uids.intersection(PubNum_to_PQNum.values())))      #401 differences everywhere
print(len(MTFC_uids.intersection(PubNum_to_PQNum.values())))          #0   none
print(set(clean_PubNum_to_PQNum.values()).difference(PubNames_uids))  # that one ?weird? item

In [ ]:
# nnnnnnn format  -- answer:  all unique

print(len(DTLSU_Restrictions_uids.intersection(DTLSU_from_microfilm_uids)))           #0
print(len(DTLSU_Restrictions_uids.intersection(PubNum_to_PQNum.keys())))              #0
print(len(DTLSU_from_microfilm_uids.intersection(PubNum_to_PQNum.keys())))            #0

In [ ]:
# overlap of setQ's marc record and the excel files' EPnnn format

print(len(marc_images_uids))                                         # 441 is good
print(len(all_set_Q_EPnnn))                                          # 441 + 1 weird
print(len(all_set_Q_EPnnn.intersection(marc_images_uids)))           # 441 is good result
print(all_set_Q_EPnnn.symmetric_difference(marc_images_uids))        # one weird item

In [ ]:
def quat_text(text):
    new_text = ''.join(i for i in text if i.isalpha())
    return new_text.lower()

In [ ]:
this_sheetname = 'DigitizeFromMicrofilm'

all_odds_titles = set(quat_text(clean_title(v['Title'])) for v in all_odds_dict['b'][this_sheetname].values())
marc_all_records_titles = set(quat_text(lookup_clean_title(record)) for record in marc_images_records)

intersec = all_odds_titles.intersection(marc_all_records_titles)
print(len(all_odds_titles), len(marc_all_records_titles), len(intersec))

not_yet_ingested_pub_nums = [int(v['Pub_Num'])
                             for v in all_odds_dict['b'][this_sheetname].values()
                             if quat_text(clean_title(v['Title'])) not in intersec]

already_ingested_pub_nums = [int(v['Pub_Num'])
                             for v in all_odds_dict['b'][this_sheetname].values()
                             if quat_text(clean_title(v['Title'])) in intersec]


count = 0
for i in all_odds_titles:
    if 'anguishedamericaneaster' in i:
        print(i)

print('***')

count = 0
for i in marc_all_records_titles:
    if 'anguishedamericaneaster' in i:
        print(i)
        
print(len(marc_all_records_titles))
print(len(not_yet_ingested_pub_nums))
print(len(already_ingested_pub_nums))


In [ ]:
# exporting the matches & non-matches as csvs

list_of_item_dicts = [k for k in all_odds_dict['b'][this_sheetname].values()]
records_list = [['Pub_Num', 'Title', 'Author ', 'Subjects', 'Degree_Type', 'Degree_Year', 'Diss_Type', 'School_Name', 'Notes', 0],]
for item in list_of_item_dicts:
    if quat_text(clean_title(item['Title'])) in intersec:
        records_list.append([item[k] for k in records_list[0]])
print(records_list)

# csv_writer(records_list, '/home/francis/Desktop/{}.csv'.format(this_sheetname))

## Editing MARC & moving pdfs

In [ ]:
# from pymarc import MARCWriter

# trash_file = '/home/francis/Desktop/trash.marc'

# # actual_restricted_records = [i for i in orig_restricted_records if lookup_uid(i) not in all_restricteds]
# # actual_restricted_records.extend([i for i in orig_unrestricted_records if lookup_uid(i) in restricted_uids])

# print(len(expected_unrestricted_records))


# with open(trash_file, 'wb') as f:
#     for record in expected_unrestricted_records:
#         f.write(record.as_marc())
    
# with open(trash_file, 'rb') as f:
#     reader = MARCReader(f)
#     trash_records = list()
#     for record in reader:
#         trash_records.append(record)
#     print(len(trash_records), len(expected_unrestricted_records))

# print(expected_unrestricted_records[0].as_dict())

In [ ]:
# import shutil

# for uid in folder_duplicated:
#     source = os.path.join(folder_on_U, 'UnrestrictedTheses', '{}.pdf'.format(uid))
#     dest = os.path.join(folder_on_U, 'TitlesDuplicatedInETDDatbase', '{}.pdf'.format(uid))
#     if os.path.isfile(source):
#         print(source, '\n', dest)
#         shutil.move(source, dest)

        
# for uid in folder_true_restrict:
#     source = os.path.join(folder_on_U, 'UnrestrictedTheses', '{}.pdf'.format(uid))
#     dest = os.path.join(folder_on_U, 'RestrictedTheses', '{}.pdf'.format(uid))
#     if os.path.isfile(source):
#         print(source, '\n', dest)
#         shutil.move(source, dest)

In [ ]:
# set_a = set()
# for record in marc_unrestricted_records:
#     set_a.add(match_degrees(record)[-6:-1])

In [ ]:
# set_a

In [ ]:
# for item in marc_images_records:
#     uid = lookup_uid(item)
#     if uid in marc_unrestricted_uids:
#         print("{}: dupe in unrestricted".format(uid))
#     if uid in marc_restricted_uids:
#         print("{}: dupe in restricted".format(uid))
#     else:
#         print('{}: not found dupe'.format(uid))

In [ ]:
# for item in marc_images_records:
#     for k,v in item.as_dict().items():
#         print(k, v)
#     break

In [ ]:
starting_title = """Syntheses and properties of isoporphyrins and related derivatives for application in photodynamic therapy"""
final_title = """Synthesis and Properties of Isoporphyrins and Related Derivatives for Application in Photodynamic Therapy"""

print(quat_text(starting_title) == quat_text(final_title))
print(quat_text(starting_title))
print(quat_text(final_title))

In [ ]:
existing_dept_full = dict()
for row, value in all_odds_dict['d']['Department Codes'].items():
    if existing_dept_full.get(value['DEPT']):
        print('oops')
    existing_dept_full[value['DEPT']] = value['department']
    
print(len(existing_dept_full))

In [ ]:
for row, value in all_odds_dict['d']['Sheet1'].items():
    if not existing_dept_full.get(value['DEPT']):
        print(value['DEPT'])
existing_dept_full['MRSC'] = 'MRSC'

In [ ]:
for update_row, update_value in all_odds_dict['e']['Sheet1'].items():
    match = False
    for original_row, original_value in all_odds_dict['f']['8734465'].items():
        if update_value['author1_lname'] == original_value['author1_lname'] and \
            update_value['author1_mname'] == original_value['author1_mname'] and \
            update_value['author1_fname'] == original_value['author1_fname']:
                if existing_dept_full[update_value['DEPT']] != original_value['department']:
                    print(original_value['title'])
                    print(existing_dept_full[update_value['DEPT']])
                    print(original_value['department'])
                    print(original_value['legacy_department'])
                    print('\n')
                match = True
    if not match:
            print(update_value['author1_fname'], update_value['author1_mname'], update_value['author1_lname'])
#             print(original_value['author1_fname'], original_value['author1_mname'], original_value['author1_lname'])